In [129]:
import pandas as pd
import datetime as datetime
# 导入因子函数库
from jqfactor import get_factor_values
# 导入基本财务信息函数库
from jqdata import *
import warnings
import math
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('stragy_momentum')

In [130]:
class private_tools():
    def compare_serice(self, serice_1, serice_2, flag):
        if flag == 'max':
            compare_flag = (serice_1 > serice_2)
            serice_2[compare_flag] = serice_1[compare_flag]
            return serice_2
        if flag == 'min':
            compare_flag = (serice_1 < serice_2)
            serice_2[compare_flag] = serice_1[compare_flag]
            return serice_2
tools = private_tools()

In [167]:
class stragy_turtleTrade():
    def __init__(self, tradeSys = None):

        self.turtle = turtleTrading(N_step = 20)
        
    def initialize(self, context):
        #setCommission(context)
        self.set_params()
        
    def set_params(self):
       
        tar_stock = '000725.XSHE'
        self.unit = None #仓位粒度
        self.unit_limit = 4 #仓位控制限制
        self.break_price = None #买入线
    
    def set_commission(self):
        dt = datetime.datetime.strptime(context.current_dt, '%Y-%m-%d')
        if dt>datetime.datetime(2013,1, 1):
            set_commission(PerTrade(buy_cost=0.0003, sell_cost=0.0013, min_cost=5)) 

        elif dt>datetime.datetime(2011,1, 1):
            set_commission(PerTrade(buy_cost=0.001, sell_cost=0.002, min_cost=5))

        elif dt>datetime.datetime(2009,1, 1):
            set_commission(PerTrade(buy_cost=0.002, sell_cost=0.003, min_cost=5))

        else:
            set_commission(PerTrade(buy_cost=0.003, sell_cost=0.004, min_cost=5))
            
    def before_trading(self):
        #set_slippage(FixedSlippage(0)) #设置滑点
        #setCommission() #设置手续费
        print('test:',self.context.current_dt)
        current_date = datetime.datetime.strptime(self.context.current_dt, '%Y-%m-%d')
        history_data = get_price(self.tar_stock, end_date = current_date, frequency='daily', fields=('high','low','close'), skip_paused=True, fq='pre', 
                      count = self.N_step, panel=False)
        self.turtle.get_volatility_N(history_data) #计算加权移动波动
        self.turtle.get_tqa_line(history_data) #计算唐奇安通道上、中、下轨
        
    def handle(self):
        context = self.context
        account_value = context.portfolio.portfolio_value #获取账户市值
        available_cash = context.portfolio.cash #账户现金余额
        volatility = self.turtle.volatility[-1] * 1
        self.unit = (value * 0.01) / volatility #当前波动下，最大单次损失1%的购买量

        #current_dt = context.current_dt
        #price_data = get_price(tar_stock, end_date = current_dt, frequency = '1m', count=1)    
        price_data = get_price(tar_stock, end_date='2020-06-01', frequency = '1m', count=1)
        if context.portfolio.positions == 0:
            market_in(context, price_data['close']) #开仓买入
        else:
            stop_loss(context, price_data['close']) #止损
            stop_gain(context, price_data['close']) #止赢
            market_add(context, price_data['close']) #加仓
            market_out(context, price_data['close']) #出局
    
    '''利用唐奇安通道，上传上轨突破视为开仓信号'''
    def market_in(self, context, curent_price):    
        # 当前价格突破唐奇安通道上轨
        if current_price > self.turtle.tqa_high[-1]:
            available_cash = context.portfolio.available_cash        
            tobuy_shares = available_cash / current_price #计算当前可买量
            if tobuy_shares >= self.unit and context.portfolio.positions[tar_stock]< int(self.unit_limit * self.unit):
                log.info('开仓买入股票数量：{}'.format(self.unit))
                order_info = order(tar_stock, self.unit)
                if order_info:
                    if str(order_info.status) == 'S':
                        log.info('下单成功！！')
                        break_price = current_price
                        
    '''再买入价基础上，股价突破break_price + 0.5*volatility，视为有效突破'''
    def market_add(self, context, curent_price):    
         if current_price >= (break_price + 0.5 * self.turtle.volatility[-1]):
            cash = context.portfolio.available_cash
            tobuy_shares = cash / current_price
            if tobuy_shares >= unit and context.portfolio.positions[tar_stock] < (self.unit_limit * self.unit):            
                order_info = order(tar_stock, self.unit)
                if order_info:
                    if str(order_info.status) == 'S':
                        log.info('下单成功！！')
                        break_price = current_price
    '''股价跌破唐奇安中轨，视为打开下降通道，需要全部卖出'''
    def market_out(self, context, curent_price):    
         if current_price >= self.turtle.tqa_middle[-1]:
            log.info('跌破中轨，离场')
            order_info = self.tradeSys.order_target(tar_stock, 0)
            if order_info:
                if str(order_info.status) == 'S':
                    log.info('清仓成功！！')
                    
    '''股价回调至最后买入价下方两个单位的波动，则清仓止损'''                      
    def stop_loss(context,current_price):
        if current_price < (break_price - 2 * self.turtle.volatility[-1]):
            order_info = self.tradeSys.order_target(g.security, 0)
            if order_info:
                if str(order_info.status) == 'S':
                    log.info('清仓成功成功！！')

In [168]:
class turtleTrading():
    
    def __init__(self, N_step = 20):
        self.tqa_high = []
        self.tqa_middle = []
        self.tqa_low = []
        self.N_compute = []
        self.N_step = N_step #默认10
        
    def compute_volatility_N(data = None):
        if not data:
            print('未发现待处理数据！')
            return None  
        hign_close_max = data['high'] - data['low']
        high_closepre_max = abs(data['high'] - data['close'].shift(1).fillna(0))#绝对值
        low_closepre_max = abs(data['low'] - data['close'].shift(1).fillna(0))#绝对值
        middle_state = tools.compare_serice(hign_close_max[1:], high_closepre_max[1:], 'max')
        volatility_range = tools.compare_serice(middle_state, low_closepre_max[1:], 'max')
        return volatility_range
        
    def get_volatility_N(data = None):
        if not data:
            print('未发现待处理数据！')
            return None
        if len(N_compute) == 0:    
            volatility_range = compute_volatility_N(data)
            self.N_compute.append(volatility_range) 
        else:
            volatility_range = compute_volatility_N(data)
            #N=Rolling((PreN∗19+TrueRange20) / 20)
            volatility_range_weight = (N_compute[-1]*(self.N_step-1) + volatility_range*(self.N_step)) / self.N_step
            self.N_compute.append(volatility_range_weight.rolling(window = 2,min_periods = 1).mean())#移动平均
            
    def get_tqa_line(data = None):

        if not data:
            print('未发现待处理数据！')
            return None
        self.tqa_high.append(max(data['high']))
        self.tqa_low.append(min(data['low']))
        self.tqa_middle.append(((max(data['high']) + min(data['low']))/2.0))


In [171]:
class backTestSimple():
    def __init__(self, stragy = None, tradeSys = tradeSystemSimple(), start_date = None, end_date = None, frequency = '1d', 
                 risk_free_rate = 0.02, benchmark = None, init_cash = None):
        self.benchmark = benchmark
        self.risk_free_rate = risk_free_rate
        self.start_date = start_date
        self.end_date = end_date
        self.frequency = frequency        
        tradeSys.init_cash = init_cash
        self.tradeSys = tradeSys
        self.stragy_m = stragy
        self.index = '000001.XSHG'
    
    '''回测驱动'''    
    def start(self, context):
        #获取回测频率
        stragy_m = self.stragy_m
        df_data = get_price(self.index, start_date = self.start_date, end_date=self.end_date, frequency = self.frequency)
        trade_frequency = df_data.index
        for trade_delta in trade_frequency:
            context.current_dt = trade_delta
            print('test1:{}'.format(context))
            stragy_m.initialize(context)
            stragy_m.before_trading(context)
            stragy_m.handle(context)
            stragy_m.after_trading(context)
        self.estimate()   
        self.visualize()
        
    
    '''回测可视化'''
    def visualize(self):
        df = pd.DataFrame(self.gain_recards, columns=['date', 'total_value', 'nav', 'bench_price'])
        df.set_index('date', inplace=True)
        df.index = pd.to_datetime(df.index)
        df['bench_nav'] = df['bench_price'] / benchmark_datadf['bench_price'][0]
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['nav'], color='pink', label='策略收益')
        plt.plot(df.index, df['bench_nav'], color='red', label='基准收益')
        plt.axhline(0,ls='--')
        plt.title('回测收益曲线图')
        plt.xlabel('时间（单位：天）')
        plt.ylabel('收益')
        plt.legend()

    
    '''策略评估目前产出：最大回撤，夏普比率，胜率，beta，alpha，待补充'''
    def estimate(self):
        df = pd.DataFrame(self.gain_recards, columns=['date', 'total_value', 'nav', 'bench_price'])
        df.set_index('date', inplace=True)
        df.index = pd.to_datetime(df.index)
        max_down_rate, max_down_start, max_down_end = self.get_max_down(df)
        sharp_ratio = self.get_sharp_ratio(df)
        win_ratio = self.get_win_ratio(df)
        beta = self.get_Beta(df)
        alpha = self.get_alpha(df)        
    
    '''获取最大回撤'''       
    def get_max_down(self, df):
      
        df['max_nav'] = df['nav'].expending().max()
        df['down_rate'] = df['nav'] / df['max_nav']
        max_down_rate = df['down_rate'].sort_values(['down_rate'], 
                                ascending = True).iloc[[0],df.columns.get_loc('down_rate')]
        min_down_rate = df[df.index < max_down_rate.index[0]].sort_values(['down_rate'], 
                                ascending = False).iloc[[0],df.columns.get_loc('down_rate')]
        min_down = df[df['max_dows_rate'] < max_down].sort_values(['max_dows_rate'], 
                                ascending = False).iloc[[0],df.columns.get_loc('max_dows_rate')]       
        max_down = 1 - max_down_rate.values #最大回撤
        max_down_start = min_down.index[0] #最大回撤开始日期
        max_down_end = max_down.index[0] #最大回撤结束日期
        return max_down, max_down_start, max_down_end
        
    def get_sharp_ratio(self, df):
        '''
         夏普率衡量的是风险和收益的平衡
         使用过程中，分为两种，一种是事先夏普率，另一种是事后夏普率，
         事先夏普率：组合收益率、无风险收益率、组合波动率是预期数据。
         事后夏普率：组合收益率、无风险收益率、组合波动率是历史数据。
         事后夏普率计算方式有不同的标准，其时间频率是年华夏普比率。
         方式一：（日均收益率-无风险收益率）/ 组合标准差   * sqr（252）
         方式二：超额收益的日均值 / 组合标准差   * sqr（252）
         本方法采用第二种方式
        '''
        #超额收益
        df['excess_income'] = df['nav'].pct_change().fillna(0.0) - df['bench_price'].pct_change().fillna(0.0)
        excess_income_mean = df['excess_income'].mean()
        excess_income_volatility =  df['excess_income'].std()
        return excess_income_mean.div(excess_income_volatility) * np.sqrt(252)
    
    def get_win_ratio(self, df):
        
        df['income_rate'] = df['nav'].pct_change().fillna(0)
        win_num = df[df['income_rate'] > 0].shape[0]
        return win_num / df.shape[0]
       
    def get_Beta(self, df):
        '''
        计算Beta值的方式有两种
        种类一：公式法，Cov（标的，基准）/ Var（基准）
        种类二：线性回归，标的收益 = Rf + beta * （标的收益 - 基准收益）
        本方法采用公式法
        '''
        df['income_rate'] = df['nav'].pct_change().fillna(0.0)
        df['bench_rate'] = df['bench_price'].pct_change().fillna(0.0)
        return np.cov(df['income_rate'], df['bench_rate']) / np.var(df['bench_rate'])
    
    def get_alpha(self, df):
        beta = self.get_Beta(df)
        a_return = self.aunual_return(df)
        bench_return = self.aunual_beach_return(df)
        return (a_return - self.risk_free_rate - beta(bench_return - self.risk_free_rate))
        
    def aunual_return(self, df):
        stragy_return = df['nav'][-1:]/df['nav'][0] -1
        a_return = ((stragy_return + 1) ** (250/df.shape[0]) -1)
        return a_return
    
    def aunual_beach_return(self, df):
        stragy_return = df['bench_price'][-1:]/df['bench_price'][0] -1
        a_return = ((stragy_return + 1) ** (250/df.shape[0]) -1)
        return a_return
        
             

In [147]:
class tradeSystemSimple():
    def __init__(self, init_cash = None):
        '''创建账户数据结构'''
        self.init_cash = init_cash
        #账户持仓情况，字典数据类型;可用余额;市值
        self.context = {'portfolio':{'positions':{},'avaible_cash':self.init_cash,'total_value':self.init_cash},
                  'current_dt':None}
        self.context = pd.DataFrame(context)
        
        '''创建与下单相关数据结构'''
        self.order_info = {}
        self.trade_recards = []
        self.gain_recards = []
        
    '''按目标股数下单,tar_stock_num为目标持股数据量'''
    def order_target(self, stock, tar_stock_num):
        order_info = {} #存放下单状态
        hold_stock_num = self.context.portfolio.positions[stock]
        stock_price = get_price(stock, end_date = self.context['current_dt'], frequency = self.frequency)['close']
        if tar_stock_num == 0:    
            sail_stock_num = self.context.portfolio.positions[stock]
            self.context.portfolio.avaible_cash += self.context.portfolio.positions[stock] * stock_price
            self.context.portfolio.positions[stock] = 0
            logger.info('清仓成功，卖出股票:{}:股票数量{},股票价格:{}'.format(stock, self.context.portfolio.positions[stock],
                                                             stock_price))
            self.record_trade(stock, 'S', sail_stock_num)
            order_info['status'] = 'S'
            return order_info
        elif (hold_stock_num > tar_stock_num):
            sail_stock_num = math.floor((scalehold_stock_num - tar_stock_num)/100) * 100
            self.context.portfolio.positions[stock] -= sail_stock_num
            self.context.portfolio.avaible_cash += sail_stock_num * stock_price
            logger.info('下单成功，卖出股票:{}:股票数量{},股票价格:{}'.format(stock, sail_stock_num,
                                                             stock_price))
            self.record_trade(stock, 'S', sail_stock_num)
            order_info['status'] = 'S'
            return order_info
        else:
            buy_stock_num = math.floor((tar_stock_num - scalehold_stock_num)/100) * 100
            self.context.portfolio.positions[stock] += buy_stock_num
            self.context.portfolio.avaible_cash -= buy_stock_num * stock_price
            logger.info('下单成功，买入股票:{}:股票数量{},股票价格:{}'.format(stock, buy_stock_num,
                                                             stock_price))
            self.record_trade(stock, 'B', buy_stock_num)
            order_info['status'] = 'S'
            return order_info
        
    def order(self, stock, amount):
        order_info = {} #存放下单状态
        #获取股票价格
        stock_price = get_price(stock, end_date = self.context['current_dt'], frequency = self.frequency)['close']
        buy_stock_num = math.floor((amount / stock_price) / 100) * 100
        if buy_stock_num > 0:
            self.context.portfolio.positions[stock] += buy_stock_num
            self.context.portfolio.self.context.portfolio -= buy_stock_num * stock_price
            logger.info('下单成功，买入股票:{}:股票数量{},股票价格:{}'.format(stock, buy_stock_num,
                                                             stock_price))
            self.record_trade(stock, 'B', buy_stock_num)
            order_info['status'] = 'S'
            return order_info
        else:
            logger.info('资金不足，下单失败')
            order_info['status'] = 'F'
            return order_info
    
    def total_value(self):
        self.context.portfolio.total_value =self.context.portfolio.avaible_cash
        for stock in self.context.portfolio.positions.keys():
            stock_price = get_price(stock, end_date = self.context['current_dt'], frequency = self.frequency)['close']
            self.context.portfolio.total_value += self.context.portfolio.positions[stock] + stock_price
        return self.context.portfolio.total_value  
    
    '''当前频率结束，记录每个交易日账户情况'''
    def record_trade(self, stock, signal, stock_num):
        nav = round((self.total_value() / self.init_cash), 2) # 计算持仓净值
        self.trade_records.append((self.context.current_dt, stock, signal, stock_num, self.total_value(), nav))
        if((self.context.current_dt.hour == 15) and (context.current_dt.minute == 0)):
            nav = round((self.total_value() / self.init_cash), 2) # 计算持仓净值
            current_date = datetime.datetime.strptime(self.context.current_dt, '%Y-%m-%d')
            bench_price = 0.00
            if self.benchmark:
                bench_price = get_price(self.benchmark, end_date = current_date)['close']
            else:
                bench_price = get_price(stock, end_date = current_date)['close']
            self.gain_recards.append((current_date, self.total_value(), nav, bench_price))
            
    '''获取回测过程中交易记录'''
    def get_records(self):
        df = pd.DataFrame(self.trade_records, columns=['date', 'stock', 'signal', 'total_value', 'nav'])
        df.set_index('date', inplace=True)
        df.index = pd.to_datetime(df.index)
        return df
              

In [172]:
tradeSys = tradeSystemSimple()
stragy = stragy_turtleTrade(tradeSys)
backTest = backTestSimple(stragy = stragy, start_date = '2019-01-01', end_date = '2020-01-01',frequency = '1m',  
                          init_cash = 1000000)


In [173]:
backTest.start()

test1:             portfolio          current_dt
avaible_cash       100 2019-01-02 09:31:00
positions           {} 2019-01-02 09:31:00
total_value        100 2019-01-02 09:31:00
test: avaible_cash    None
positions       None
total_value     None
Name: current_dt, dtype: object


TypeError: strptime() argument 1 must be str, not Series

In [100]:
df_data = get_price(tar_stock, end_date='2020-06-01', frequency = '1m')
#pd.DataFrame(df_data.index).dtypes
print(df_data.index[1])
df_data = get_price(tar_stock, end_date=df_data.index[1], frequency = '1m', count=1)
df_data

2015-01-05 09:32:00


,open,close,high,low,volume,money
2015-01-05 09:32:00,3.27,3.27,3.28,3.26,5217441.0,17074392.0


In [98]:
df_data = attribute_history(tar_stock,10,'1d',('close','low'))
df_data['low'][1]*100

383.0

In [99]:
order_info = None
order_info['000735'] = 1
order_info

TypeError: 'NoneType' object does not support item assignment

In [96]:
data = {}
positions = {}
portfolio = {}
data['portfolio'] = portfolio
portfolio['positions'] = positions
context = pd.DataFrame(context)

In [97]:
positions['000735'] = 1
data.portfolio.positions['000735']

1

In [94]:
positions['000735'] = 1
context['portfolio']['positions']['000735']

1

In [ ]:
context = {}
positions = {}
portfolio = {}
positions['000735'] = 100
portfolio['positions'] = positions
context['portfolio'] = portfolio

In [108]:
test = pd.DataFrame(np.arange(10).reshape(2,5))
np.cov(test[0])

array(12.5)

In [112]:
test_expand = test.expanding(min_periods=1,axis=0).mean()
test_expand.columns = ['one','two','three','four','five']
test_expand

,one,two,three,four,five
0,0.0,1.0,2.0,3.0,4.0
1,2.5,3.5,4.5,5.5,6.5


In [116]:
test_expand['four'][-1]/test_expand['four'][0]

KeyError: -1

In [179]:
class tes1():
    def __init__(self):
        self.start_date = '2015-02-03'
        print(id(self.start_date))
class test():
    def __init__(self, start_date):
        self.start_date = start_date
        self.a = None
    def start(self):
        a = 10
        print(id(self.start_date))
        print(self.start_date)
tes1 = tes1()
print(id(tes1.start_date))
test = test(tes1.start_date)
test.start()

140369697875952
140369697875952
140369697875952
2015-02-03


In [180]:
tes1.start_date  = '206-0609'
print(id(tes1.start_date))
test.start()

140369722342640
140369697875952
2015-02-03


In [ ]:
stock_df['max_total'] = stock_df['total'].expanding().max()
#计算资金曲线在滚动最高值之后所回撤的百分比
stock_df['per_total'] = stock_df['total']/stock_df['max_total']
min_point_total = stock_df.sort_values(by=['per_total']).iloc[[0], stock_df.columns.get_loc('per_total')]
print(min_point_total)
max_point_total = stock_df[stock_df.index <= min_point_total.index[0]].sort_values\         
(by=['total'],ascending=False).iloc[[0],stock_df.columns.get_loc('total')]
print("最大资金回撤%5.2f%%从%s开始至%s结束"%((1-min_point_total.values),\
max_point_total.index[0],min_point_total.index[0]))